# Imports and Stuff

In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_lg")
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

print("done")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\laura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\laura\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


done


In [14]:
from keras.models import load_model
model = load_model('chatbot_model_spacy.h5')
import json
import random
import pandas as pd
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
intents = json.loads(open('inquiries.json').read())
#words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))#

print("done")

done


In [5]:
movies = pd.read_csv("movies_2.tsv",sep="\t")
movies.set_index("movie", inplace=True)
movies.head(20)

,title,date,starttime,endtime,when,duration,type,actors,director,synopsis,recommendation
movie,,,,,,,,,,,
brave,Only the Brave,Saturday,20:00,22:30,Saturday at 8,2 hours and 20 minutes,drama,"Josh Brolin , Miles Teller , Jeff Bridges",Joseph Kosinski,"In 2007 Prescott, Arizona, Eric Marsh of the P...",recommend
goodbye,The Goodbye Girl,Friday,19:30,21:00,Friday at 7:30,1 hour and 17 minutes,romantic comedy,"Richard Dreyfuss , Marsha Mason , Quinn Cummings",Herbert Ross,"After being dumped by her live-in boyfriend, a...",recommend
king,The King and I,Sunday,18:45,20:30,Sunday at 18:45,1 hour and 32 minutes,drama,"Yul Brynner , Deborah Kerr , Rita Moreno",Walter Lang,Musical about a widow who accepts a job as a l...,not recommend
kills,Some Guy Who Kills People,Friday,15:45,18:30,Friday at 15:45,2 hours and 58 minutes,horror comedy,"Kevin Corrigan , Barry Bostwick , Karen Black",Jack Perez,"Ken Boyd, a lonely man fresh out of the loony ...",not recommend
ragamuffin,Ragamuffin,Monday,19:50,21:00,Monday at 19:50,1 hour and 38 minutes,biography,"Michael Koch , David Leo Schultz , Mel Fair , ...",David Leo Schultz,"Based on the life of Rich Mullins, a musical p...",recommend
secrets,Official Secrets,Tuesday,19:00,21:30,Tuesday at 19:00,2 hours and 12 minutes,thriller,"Keira Knightley , Matthew Goode , Ralph Fienne...",Gavin Hood,The true story of a British whistleblower who ...,recommend
ghost,A Ghost Story,Thurday,16:00,18:00,Thursday at 16:00,1 hour and 46 minutes,fantasy,"Casey Affleck , Rooney Mara , McColm Cephas Jr",David Lowery,Resonating with vibrant memories and silent ec...,recommend
coldwater,Coldwater,Saturday,24:00,2:30,Saturday at 24:00,2 hours and 13 minutes,thriller,"PJ Boudousque , James C Burns , Chris Petrovski",Vincent Grashaw,A teenage boy is sent to a juvenile reform fac...,not recommend
animal,Animal Factory,Friday,20:15,22:30,Friday at 20:15,2 hours,drama,"Willem Dafoe , Edward Furlong , Danny Trejo",Steve Buscemi,"A young man goes to prison and a tough, older ...",recommend


# Random Tests

In [37]:
#Match movie

movie_matcher_phrase = PhraseMatcher(nlp.vocab, attr="LOWER")
movie_matcher = Matcher(nlp.vocab)
for movie, row in movies.iterrows():
    # match whole phrase
    title = nlp(row["title"])
    patterns  = [title]
    movie_matcher_phrase.add(movie, None, *patterns)
    # match description
    nouns = []
    for token in title:
        if token.pos_ in ["NOUN", "PROPN"]:
            nouns.append(token.text.lower())
    print(row["title"])
    print(nouns)
    if nouns:
        movie_matcher.add(movie, None, [{"lower": "the", "OP": "*"},{"lower": {"IN": ["one", "movie", "film", "picture"]}}, {"lower": {"IN": ["with", "about"]}}, {"lower": {"IN": ["the", "that", "a"]}, "OP": "*"}, {"lower": {"IN": nouns}}])
    
print("done")

Only the Brave
[]
The Goodbye Girl
['goodbye', 'girl']
The King and I
['king']
Some Guy Who Kills People
['guy', 'people']
Ragamuffin
['ragamuffin']
Official Secrets
['secrets']
A Ghost Story
['ghost', 'story']
Coldwater
['coldwater']
Animal Factory
['animal', 'factory']
Ice Age V
['ice', 'age', 'v']
The Book of Life
['book', 'life']
done


In [44]:
chat = True
while chat:
    msg = input().strip()
    if msg=="stop": 
        chat = False
        break
    doc = nlp(msg)
    matches = movie_matcher_phrase(doc)
    matches += movie_matcher(doc)
    if matches:
        match_length = 0
        movie_match_id = -1
        movie_start = -1
        movie_end = -1
        for match_id, start, end in matches:#
            span = doc[start:end]
            print(span, start, end, "current best length: ", match_length)
            #print(match_length, start, end)
            if (end-start)>match_length:
                print("new best length!", end-start)
                match_length = end-start
                movie_match_id = match_id
                movie_start = start
                movie_end = end
        movie = nlp.vocab.strings[movie_match_id]
        span = doc[movie_start:movie_end]
        msg_without_start = doc[:movie_start].text
        msg_without_end = doc[movie_end:].text
        msg_without = msg_without_start + " it " + msg_without_end
        print(msg)
        print(span.text)
        print(msg_without)
    

Tell me the plot of the movie about the girl?
the movie about the girl 5 10 current best length:  0
new best length! 5
movie about the girl 6 10 current best length:  5
Tell me the plot of the movie about the girl?
the movie about the girl
Tell me the plot of it ?
Which actors are in the one about that king
the one about that king 4 9 current best length:  0
new best length! 5
one about that king 5 9 current best length:  5
Which actors are in the one about that king
the one about that king
Which actors are in it 
stop


In [6]:
import simplenlg
from simplenlg.framework import *
from simplenlg.lexicon import *
from simplenlg.realiser.english import *
from simplenlg.phrasespec import *
from simplenlg.features import *

lexicon = Lexicon.getDefaultLexicon()
nlgFactory = NLGFactory(lexicon)
realiser = Realiser(lexicon)

p = nlgFactory.createClause()
p.setSubject("it")
p.setVerb("be")
p.setObject("romance")
p.setFeature(Feature.NEGATED, True)

output2 = realiser.realiseSentence(p)
print(output2)

It is not romance.


In [ ]:
from spacy import displacy

chat = True
while chat:
    msg = input().strip()
    if msg=="stop": 
        chat = False
        break
        
    p = nlgFactory.createClause()
        
    doc = nlp(msg)
    
    if doc[0].pos_=="AUX" or doc[0].dep_=="aux":
        for token in doc:
            if token.dep_ == "ROOT":
                p.setVerb(token.text)
                for child in token.children:
                    if child.dep_ == "nsubj":
                        if child.lower_ == "i": subject = "you"
                        elif child.lower_ == "you": subject = "i"
                        else: subject = child.text
                        p.setSubject(subject)
                    if child.dep_ == "dobj" or child.dep_ == "ccomp":
                        rest = ""
                        for grand_child in child.children:
                            if grand_child.lower_ == "i": rest += " you"
                            elif grand_child.lower_ == "you": rest += "I"
                            else: rest += grand_child.text
                        print(rest)
                        #print(child.text)
                        #p.setObject(child.text)
                        #for chunk in doc.noun_chunks:
                         #   if chunk.root == child:
                          #      print(chunk.text)
                           #     p.setObject(chunk.text)
        #p.setFeature(Feature.NEGATED, True)
        
    print("\t - ",realiser.realiseSentence(p),"\n")
            
    displacy.render(doc, style='dep')
            



Do you think I should see the movie?
 youshouldmovie
	 -  I thinks. 



In [47]:
def get_question_type(doc):
    
    verb = False
    verbs = []
    noun = False
    nouns = []
    adj = False
    adjs = []
    bad_words = ["how","when"]
    
    for token in doc:
        print(token.text, token.lemma_, token.pos_, token.dep_, "\n")
    
    if doc[0].dep_ == "aux" and doc[1].text == "you" and doc[2].pos_ == "VERB":
        if doc[3].text == "me": doc = doc[4:]
        else: doc = doc[3:]
        print("yes")
        
        
    
    for token in doc:
        if token.pos_ in ["VERB", "AUX"]:
            verb = True
            verbs.append(token.lemma_)
        if token.pos_ in ["ADJ", "ADV"] and not token.lemma_ in bad_words:
            adj = True
            adjs.append(token.lemma_)
        if token.pos_ in ["NOUN", "PROPN"]:
            noun = True
            nouns.append(token.lemma_)
    result = [["verbs",verb,verbs],["nouns",noun,nouns],["adjs",adj,adjs]]
    for category in result: print(category)
    return (result)

In [49]:
chat = True
while chat:
    msg = input().strip()
    if msg=="stop": 
        chat = False
        break
        
    doc = nlp(msg)
    for token in doc: print(token.text, token.pos_)
    get_question_type(doc)

When does taken start?
When ADV
does AUX
taken VERB
start VERB
? PUNCT
When when ADV advmod 

does do AUX aux 

taken take VERB ROOT 

start start VERB dobj 

? ? PUNCT punct 

['verbs', True, ['do', 'take', 'start']]
['nouns', False, []]
['adjs', False, []]
stop


In [48]:
with open("example_questions.txt") as fp:
    line = fp.readline()
    while line:
        line = fp.readline()
        doc = nlp(line)
        print(line)
        get_question_type(doc)
        #displacy.render(doc, style='dep')
        print("\n")

What's its runtime?

What what PRON attr 

's be AUX ROOT 

its -PRON- DET poss 

runtime runtime NOUN attr 

? ? PUNCT punct 


 
 SPACE  

['verbs', True, ['be']]
['nouns', True, ['runtime']]
['adjs', False, []]


Can you look up the synopsis of Taken?

Can Can VERB aux 

you -PRON- PRON nsubj 

look look VERB ROOT 

up up ADP prt 

the the DET det 

synopsis synopsis NOUN dobj 

of of ADP prep 

Taken take VERB pobj 

? ? PUNCT punct 


 
 SPACE  

yes
['verbs', True, ['take']]
['nouns', True, ['synopsis']]
['adjs', False, []]


When will it run?

When when ADV advmod 

will will VERB aux 

it -PRON- PRON nsubj 

run run VERB ROOT 

? ? PUNCT punct 


 
 SPACE  

['verbs', True, ['will', 'run']]
['nouns', False, []]
['adjs', False, []]


Could you tell me what Taken 22 is about?

Could Could VERB aux 

you -PRON- PRON nsubj 

tell tell VERB ROOT 

me -PRON- PRON dobj 

what what PRON pobj 

Taken take VERB nsubj 

22 22 NUM dobj 

is be AUX ccomp 

about about ADJ prep 

? ? PUNCT p

IndexError: [E040] Attempt to access token at 0, max length 0.

In [56]:
import pyinflect

chat = True
while chat:
    msg = input().strip()
    if msg=="stop": 
        chat = False
        break
        
    doc = nlp(msg)
    for token in doc: 
        if token.pos_ == "VERB" or token.pos_ == "AUX":
            past = token._.inflect("VBD")
            print(token.text, token.pos_, token.dep_, token.tag_, past)

When does it start?
does AUX aux VBZ did
start VERB ROOT VB started
Could you tell me?
Could VERB aux MD Coulded
tell VERB ROOT VB told
stop
